In [152]:
import pandas as pd
import nltk
import numpy as np



In [86]:
file_name = 'data/Travel-Dataset-5000--master/5000TravelQuestionsDataset.xlsx'
col_names = ['questions', 'a', 'b']
dataset = pd.read_excel(file_name, header=None, names=col_names)
print(dataset.head())
print(dataset.info())

                                           questions    a       b
0  What are the special things we (husband and me...  TTD  TTDSIG
1  What are the companies which organize shark fe...  TTD  TTDOTH
2  Is it safe for female traveller to go alone to...  TGU  TGUHEA
3  What are the best places around Cape Town for ...  TTD  TTDSIG
4  What are the best places to stay for a family ...  ACM  ACMOTH
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   a          5000 non-null   object
 2   b          5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB
None


In [87]:
dataset['a'].unique()

array(['TTD', 'TGU', 'ACM', 'TRS', 'WTH', 'FOD', 'ENT', 'TGU\n', 'TTD\n',
       '\nENT'], dtype=object)

In [88]:
dataset['b'].unique()

array(['TTDSIG', 'TTDOTH', 'TGUHEA', 'ACMOTH', 'TRSTRN', 'TGUBAN',
       'WTHTMP', 'TGUNEI', 'TTDSPO', 'ACMAPA', 'TRSOTH', 'TRSAIR',
       'TRSROU', 'TGUOTH', 'ACMRES', 'ACMBUN', 'TRSCRS', 'WTHDRC',
       'TRSRNT', 'TGUTOP', 'FODOTH', 'TGUVIS', 'FODAUT', 'TTDTRI',
       'TGUPLN', 'TRSTAX', 'WTHOTH', 'TRSBUS', 'ACMHOT', 'TGULUG',
       'FODBAK', 'TTDSPA', 'FODBRE', 'TGUATT', 'ENTCLB', 'TGUAVE',
       'TGUTEL', 'TGUCIG', 'TTDSHP', 'TRSTCD', 'ACMBEA', 'FODCOT',
       'TGUAPT', 'TRSLIC', 'TGULAU', 'TTDGYM', 'FODCAT', 'FODBAR',
       'TGUHOL', 'ENTFES', 'TGURUL', 'TGURES', 'WTHTMP\n', 'ACMCAR',
       'FODFMA', 'ENTSHW', 'TRSGAS', '\nTGULAU', 'TRSOTH\n', 'WTHSNW',
       'ENTMUS', 'ENTSPO', 'FODBAK\n', 'TRSAIR\n', 'TGUWEB', 'TRSDRV',
       'FODFCA', 'TGUCIG\n', 'ENTOTH', 'TTDOTH\n', 'WTHOTH\n', 'TTDSIG\n',
       'TGUOTH\n', 'TTDSHP\n', 'TRSROU\n', 'TTDSPO\n', '\nACMOTH',
       'ACMOTH\n', '\nWTHOTH'], dtype=object)

In [89]:
dataset['questions'][0]

'What are the special things we (husband and me) can do during a 5 day stay at Cape Town?'

In [90]:
dataset['questions'][1]

'What are the companies which organize shark feeding events for scuba divers?'

# PreProcessing

In [91]:
dataset.head()

,questions,a,b
0,What are the special things we (husband and me...,TTD,TTDSIG
1,What are the companies which organize shark fe...,TTD,TTDOTH
2,Is it safe for female traveller to go alone to...,TGU,TGUHEA
3,What are the best places around Cape Town for ...,TTD,TTDSIG
4,What are the best places to stay for a family ...,ACM,ACMOTH


In [92]:
### Lowercase

In [93]:
test_id = 1
dataset['questions'].dropna(inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   a          5000 non-null   object
 2   b          5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [94]:
dataset['questions'][test_id]

'What are the companies which organize shark feeding events for scuba divers?'

In [95]:
def lowercase(x):
    return x.lower()

dataset['questions'] = [lowercase(question) for question in dataset['questions']]
dataset['questions'][test_id]

'what are the companies which organize shark feeding events for scuba divers?'

In [96]:
import string

def remove_punctuation(x):
    return "".join([char for char in x if char not in string.punctuation])

dataset['questions'] = [remove_punctuation(question) for question in dataset['questions']]
dataset['questions'][test_id]

'what are the companies which organize shark feeding events for scuba divers'

In [97]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# from nltk import word_tokenize

# def remove_stopwords(x):
#     words = word_tokenize(x)
#     return " ".join([word for word in words if word not in stop_words])

# dataset['questions'] = [remove_stopwords(question) for question in dataset['questions']]
# dataset['questions'][test_id]
# print(stop_words)

In [98]:
# from nltk.stem.porter import PorterStemmer
# porter = PorterStemmer()
# def stemming(x):
#     filtered_words = word_tokenize(x['questions'])
#     stemmed = [porter.stem(word) for word in filtered_words]
#     return " ".join(stemmed)

# dataset['questions'] = [remove_stopwords(question) for question in dataset['questions']]
# dataset['questions'][test_id]

In [99]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()
def lemmatize(x):
    filtered_words = word_tokenize(x)
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]
    return " ".join(lemmatized)

dataset['questions'] = [lemmatize(question) for question in dataset['questions']]
dataset['questions'][test_id]


'what are the company which organize shark feeding event for scuba diver'

In [100]:
# from nltk import pos_tag
# def pos_tagging(x):
#     words = word_tokenize(x['questions'])
#     return pos_tag(words)

# Create features

# Create Dataset

# SVM - Normal train

In [ ]:
## Train test split

In [149]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = dataset['questions'],dataset['a']

tfidf = TfidfVectorizer(max_features=5000)
tfidf.fit(X)

le = LabelEncoder()
le.fit(y)
    
cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(X):
    fold += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    y_train_e = le.transform(y_train)
    y_test_e = le.transform(y_test)
    
    X_train_tfidf = Tfidf_vect.transform(X_train)
    X_test_tfidf = Tfidf_vect.transform(X_test)
    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train_tfidf,y_train_e)
    predictions_SVM = SVM.predict(X_test_tfidf)
    acc = accuracy_score(predictions_SVM, y_test_e)*100
    accuracies.append(acc)
    print("Fold - {} - {} - {}".format(fold, "SVM Accuracy Score -> ",acc))
    
print("Mean {} Std {}".format(np.mean(accuracies), np.std(accuracies)))

Fold - 1 - SVM Accuracy Score ->  - 82.8
Fold - 2 - SVM Accuracy Score ->  - 84.0
Fold - 3 - SVM Accuracy Score ->  - 81.0
Fold - 4 - SVM Accuracy Score ->  - 82.39999999999999
Fold - 5 - SVM Accuracy Score ->  - 83.2
Fold - 6 - SVM Accuracy Score ->  - 83.2
Fold - 7 - SVM Accuracy Score ->  - 84.0
Fold - 8 - SVM Accuracy Score ->  - 84.0
Fold - 9 - SVM Accuracy Score ->  - 82.19999999999999
Fold - 10 - SVM Accuracy Score ->  - 82.8


In [145]:
accuracies

[82.8,
 84.0,
 81.0,
 82.39999999999999,
 83.2,
 83.2,
 84.0,
 84.0,
 82.19999999999999,
 82.8]

In [151]:
import numpy as np

np.std([1,2])

0.5

In [ ]:
## Word2vec

In [147]:
import gensim
data = ['data']
model = gensim.models.Word2Vec(data, min_count=1, sg=1, window = 3)